In [1]:
from fastai.conv_learner import *
from fastai.dataset import *

import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

import torch as th
th.cuda.set_device(1)

/home/kownse/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
PATH = './'
TRAIN = '../input/train_v2/'
TEST = '../input/test_v2/'
SEGMENTATION = '../input/train_ship_segmentations_v2.csv.zip'
exclude_list = ['6384c3e78.jpg','13703f040.jpg', '14715c06d.jpg',  '33e0ff2d5.jpg',
                '4d4e09f2a.jpg', '877691df8.jpg', '8b909bb20.jpg', 'a8d99130e.jpg', 
                'ad55c3143.jpg', 'c8260c541.jpg', 'd6c7f17c7.jpg', 'dc3e7c901.jpg',
                'e44dffe88.jpg', 'ef87bad36.jpg', 'f083256d8.jpg'] #corrupted image

In [3]:
nw = 4   #number of workers for data loader
arch = resnet50 #specify target architecture

In [4]:
train_names = [f for f in os.listdir(TRAIN)]
test_names = [f for f in os.listdir(TEST)]
for el in exclude_list:
    if(el in train_names): train_names.remove(el)
    if(el in test_names): test_names.remove(el)
        
# train_names = train_names[:1000]
#5% of data in the validation set is sufficient for model evaluation
tr_n, val_n = train_test_split(train_names, test_size=0.05, random_state=42)

In [5]:
class pdFilesDataset(FilesDataset):
    def __init__(self, fnames, path, transform):
        self.segmentation_df = pd.read_csv(SEGMENTATION).set_index('ImageId')
        super().__init__(fnames, transform, path)
    
    def get_x(self, i):
        img = open_image(os.path.join(self.path, self.fnames[i]))
        if self.sz == 768: return img 
        else: return cv2.resize(img, (self.sz, self.sz))
    
    def get_y(self, i):
        if(self.path == TEST): return 0
        masks = self.segmentation_df.loc[self.fnames[i]]['EncodedPixels']
        if(type(masks) == float): return 0 #NAN - no ship 
        else: return 1
    
    def get_c(self): return 2 #number of classes

In [6]:
def get_data(sz,bs):
    #data augmentation
    aug_tfms = [RandomRotate(20, tfm_y=TfmType.NO),
                RandomDihedral(tfm_y=TfmType.NO),
                RandomLighting(0.05, 0.05, tfm_y=TfmType.NO)]
    tfms = tfms_from_model(arch, sz, crop_type=CropType.NO, tfm_y=TfmType.NO, 
                aug_tfms=aug_tfms)
    ds = ImageData.get_ds(pdFilesDataset, (tr_n[:-(len(tr_n)%bs)],TRAIN), 
                (val_n,TRAIN), tfms, test=(test_names,TEST))
    md = ImageData(PATH, ds, bs, num_workers=nw, classes=None)
#     md.is_multi = False
    return md

In [7]:
sz = 256 #image size
bs = 32  #batch size

md = get_data(sz,bs)
learn = ConvLearner.pretrained(arch, md, ps=0.5) #dropout 50%
learn.opt_fn = optim.Adam

/home/kownse/anaconda3/lib/python3.6/site-packages/fastai/initializers.py:6: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  if hasattr(m, 'weight'): init_fn(m.weight)


In [8]:
learn.fit(2e-3, 2)

epoch      trn_loss   val_loss   accuracy                      
    0      0.158226   0.128588   0.953261  
    1      0.157111   0.110443   0.958351                      



[0.11044308184450674, 0.958350644004657]

In [9]:
learn.unfreeze()
lr=np.array([1e-4,5e-4,2e-3])

In [10]:
learn.fit(lr, 1, cycle_len=3, use_clr=(20,8))

epoch      trn_loss   val_loss   accuracy                       
    0      0.123423   0.091847   0.966556  
    1      0.106244   0.076049   0.972268                       
    2      0.111794   0.064196   0.975488                       



[0.06419571748024258, 0.9754881595346905]

In [11]:
learn.save('Resnet50_lable_256_1')

In [12]:
sz = 384 #image size
bs = 16  #batch size

md = get_data(sz,bs)
learn = ConvLearner.pretrained(arch, md, ps=0.5) #dropout 50%
learn.opt_fn = optim.Adam
learn.unfreeze()
lr=np.array([1e-4,5e-4,2e-3])

/home/kownse/anaconda3/lib/python3.6/site-packages/fastai/initializers.py:6: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  if hasattr(m, 'weight'): init_fn(m.weight)


In [13]:
learn.load('Resnet50_lable_256_1')

In [14]:
learn.fit(lr/2, 2, cycle_len=10, use_clr=(20,8)) #lr is smaller since bs is only 32
learn.save('Resnet50_lable_384_1')

epoch      trn_loss   val_loss   accuracy                          
    0      0.145598   0.130534   0.970503  
    1      0.137501   3.395956   0.953261                           
    2      0.114289   1.661474   0.969983                           
    3      0.102656   0.077294   0.973203                           
    4      0.111209   0.080926   0.97393                            
    5      0.08562    0.094093   0.971126                           
    6      0.061778   0.544835   0.974346                           
    7      0.074077   7.50039    0.962194                           
    8      0.070988   0.056801   0.980681                           
    9      0.057523   0.110439   0.981408                           
    10     0.102727   0.061876   0.980058                           
    11     0.090529   0.060161   0.978604                           
    12     0.093602   0.192317   0.972476                           
    13     0.086963   0.067981   0.976215                   

In [15]:
sz = 768 #image size
bs = 4  #batch size

md = get_data(sz,bs)
learn = ConvLearner.pretrained(arch, md, ps=0.5) #dropout 50%
learn.opt_fn = optim.Adam
learn.unfreeze()
lr=np.array([1e-4,5e-4,2e-3])

/home/kownse/anaconda3/lib/python3.6/site-packages/fastai/initializers.py:6: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  if hasattr(m, 'weight'): init_fn(m.weight)
